# SentenceBERT+머신러닝

## SentenceBERT 모델을 이용하여 임베딩 행렬 생성

In [ ]:
### 필요한 라이브러리 설치
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=bee7717ed44f4b45decf6b771e648729549e92442de9fc5d71c8563c8cc82042
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
### 필요한 라이브러리 임폴트
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

In [ ]:
### 학습용 데이터 --> DataFrame 생성, 누락 데이터 제거

# 데이터 불러오기
file_path='/content/drive/MyDrive/KDT/딥러닝/자연어처리/ratings_train.txt'

# pd.read_csv() 함수 사용
df_train = pd.read_csv(file_path, sep='\t')
print(f'생성된 학습용 데이터프레임 확인 : \n{df_train}')

print('-'*80)

# 각 컬럼별 누락 데이터의 수 확인
num_nulls = df_train.isnull().sum()
print(f'각 컬럼별 누락 데이터의 수 : \n{num_nulls}')

print('-'*80)

# 누락 데이터가 존재하는 행 제거 --> dropna() 사용
df_train.dropna(inplace=True)
print(f'누각 데이터 제거 후 학습용 데이터프레임 확인 : \n{df_train}')

생성된 학습용 데이터프레임 확인 : 
              id                                           document  label
0        9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1        3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2       10265843                                  너무재밓었다그래서보는것을추천한다      0
3        9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4        6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
...          ...                                                ...    ...
149995   6222902                                인간이 문제지.. 소는 뭔죄인가..      0
149996   8549745                                      평점이 너무 낮아서...      1
149997   9311800                    이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?      0
149998   2376369                        청춘 영화의 최고봉.방황과 우울했던 날들의 자화상      1
149999   9619869                           한국 영화 최초로 수간하는 내용이 담긴 영화      0

[150000 rows x 3 columns]
----------------------------------------------------

In [ ]:
### 평가용 데이터 --> DataFrame 생성, 누락 데이터 제거

# 데이터 불러오기
file_path='/content/drive/MyDrive/KDT/딥러닝/자연어처리/ratings_test.txt'

# pd.read_csv() 함수 사용
df_test = pd.read_csv(file_path, sep='\t')
print(f'생성된 평가용 데이터프레임 확인 : \n{df_test}')

print('-'*80)

# 각 컬럼별 누락 데이터의 수 확인
num_nulls = df_test.isnull().sum()
print(f'각 컬럼별 누락 데이터의 수 : \n{num_nulls}')

print('-'*80)

# 누락 데이터가 존재하는 행 제거 --> dropna() 사용
df_test.dropna(inplace=True)
print(f'누락 데이터 제거 후 평가용 데이터프레임 확인 : \n{df_test}')

생성된 평가용 데이터프레임 확인 : 
            id                                           document  label
0      6270596                                                굳 ㅋ      1
1      9274899                               GDNTOPCLASSINTHECLUB      0
2      8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3      6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4      6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0
...        ...                                                ...    ...
49995  4608761          오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함      1
49996  5308387       의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO      0
49997  9072549                 그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다      0
49998  5802125     절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네      0
49999  6070594                                         마무리는 또 왜이래      0

[50000 rows x 3 columns]
-----------------------------------------------------------------------------

In [ ]:
### 학습용 데이터에 대한 인덱스 재설정 --> df.reset_index() 사용
df_train.reset_index(drop=True, inplace=True)

# 결과 확인하기
print(df_train)

              id                                           document  label
0        9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1        3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2       10265843                                  너무재밓었다그래서보는것을추천한다      0
3        9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4        6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
...          ...                                                ...    ...
149990   6222902                                인간이 문제지.. 소는 뭔죄인가..      0
149991   8549745                                      평점이 너무 낮아서...      1
149992   9311800                    이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?      0
149993   2376369                        청춘 영화의 최고봉.방황과 우울했던 날들의 자화상      1
149994   9619869                           한국 영화 최초로 수간하는 내용이 담긴 영화      0

[149995 rows x 3 columns]


In [ ]:
### 평가용 데이터에 대한 인덱스 재설정 --> df.reset_index() 사용
df_test.reset_index(drop=True, inplace=True)

# 결과 확인하기
print(df_test)

            id                                           document  label
0      6270596                                                굳 ㅋ      1
1      9274899                               GDNTOPCLASSINTHECLUB      0
2      8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3      6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4      6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0
...        ...                                                ...    ...
49992  4608761          오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함      1
49993  5308387       의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO      0
49994  9072549                 그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다      0
49995  5802125     절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네      0
49996  6070594                                         마무리는 또 왜이래      0

[49997 rows x 3 columns]


In [ ]:
### 학습용 데이터 전처리

# pattern 생성 : 한글 글자, 자음, 모음, 공백을 제외한 나머지 제거
pattern='[^가-힣ㄱ-ㅎㅏ-ㅣ ]'

# Series.str.replace(pattern, repl)
df_train.loc[:, 'document'] = df_train.loc[:, 'document'].str.replace(pattern, " ")

# 결과 확인하기
print(df_train)

<ipython-input-7-9ecd113d12e7>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df_train.loc[:, 'document'] = df_train.loc[:, 'document'].str.replace(pattern, " ")


              id                                           document  label
0        9976970                                아 더빙   진짜 짜증나네요 목소리      0
1        3819312                  흠   포스터보고 초딩영화줄    오버연기조차 가볍지 않구나      1
2       10265843                                  너무재밓었다그래서보는것을추천한다      0
3        9045019                      교도소 이야기구먼   솔직히 재미는 없다  평점 조정      0
4        6483659  사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
...          ...                                                ...    ...
149990   6222902                                인간이 문제지   소는 뭔죄인가        0
149991   8549745                                      평점이 너무 낮아서         1
149992   9311800                    이게 뭐요  한국인은 거들먹거리고 필리핀 혼혈은 착하다       0
149993   2376369                        청춘 영화의 최고봉 방황과 우울했던 날들의 자화상      1
149994   9619869                           한국 영화 최초로 수간하는 내용이 담긴 영화      0

[149995 rows x 3 columns]


In [ ]:
### 평가용 데이터 전처리

# pattern 생성 : 한글 글자, 자음, 모음, 공백을 제외한 나머지 제거
pattern='[^가-힣ㄱ-ㅎㅏ-ㅣ ]'

# Series.str.replace(pattern, repl)
df_test.loc[:, 'document'] = df_test.loc[:, 'document'].str.replace(pattern, " ")

# 결과 확인하기
print(df_test)

            id                                           document  label
0      6270596                                                굳 ㅋ      1
1      9274899                                                         0
2      8544678             뭐야 이 평점들은     나쁘진 않지만   점 짜리는 더더욱 아니잖아      0
3      6825595                   지루하지는 않은데 완전 막장임    돈주고 보기에는          0
4      6723715    만 아니었어도 별 다섯 개 줬을텐데   왜   로 나와서 제 심기를 불편하게 하죠        0
...        ...                                                ...    ...
49992  4608761          오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함      1
49993  5308387       의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따           0
49994  9072549                 그림도 좋고 완성도도 높았지만    보는 내내 불안하게 만든다      0
49995  5802125     절대 봐서는 안 될 영화   재미도 없고 기분만 잡치고   한 세트장에서 다 해먹네      0
49996  6070594                                         마무리는 또 왜이래      0

[49997 rows x 3 columns]


<ipython-input-8-f40b25261a37>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test.loc[:, 'document'] = df_test.loc[:, 'document'].str.replace(pattern, " ")


In [ ]:
### 사전 학습된 한국어 SentenceBERT 모델 생성(다운로드)
model_name='ddobokki/klue-roberta-base-nli-sts'
model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/576 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
### 학습용 데이터(document) --> 문장 임베딩 행렬 생성

# 사용자 정의 함수
# def sentence_enc(sentence):
#     return model.encode(sentence)

# sentences 생성
sentences = df_train.loc[:, 'document'].values
print(sentences)

print('-'*80)

# model.encode(sentences)
train_embeddings = model.encode(sentences)

# 결과 확인하기
print(f'학습용 전체 리뷰에 대한 임베딩 행렬의 모양 : {train_embeddings.shape}')

['아 더빙   진짜 짜증나네요 목소리' '흠   포스터보고 초딩영화줄    오버연기조차 가볍지 않구나'
 '너무재밓었다그래서보는것을추천한다' ... '이게 뭐요  한국인은 거들먹거리고 필리핀 혼혈은 착하다 '
 '청춘 영화의 최고봉 방황과 우울했던 날들의 자화상' '한국 영화 최초로 수간하는 내용이 담긴 영화']
--------------------------------------------------------------------------------
학습용 전체 리뷰에 대한 임베딩 행렬의 모양 : (149995, 768)


In [ ]:
### 학습용 데이터에 대한 임베딩 결과 저장하기

# 파일 경로 설정
train_embeddings_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/train_embeddings.npy'

# 결과 저장하기
np.save(train_embeddings_path, train_embeddings)

In [ ]:
### 학습용 데이터에 대한 임베딩 결과 불러오기

# 파일 경로 설정
train_embeddings_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/train_embeddings.npy'

# 저장된 학습용 임베딩 행렬 불러오기
loaded_train_embeddings = np.load(train_embeddings_path)

# 결과 확인하기
print(f'학습용 리뷰 전체에 대한 임베딩 행렬의 모양 : {loaded_train_embeddings.shape}')
print('-'*80)
print(f'학습용 리뷰 전체에 대한 임베딩 행렬 확인 : \n{loaded_train_embeddings}')

학습용 리뷰 전체에 대한 임베딩 행렬의 모양 : (149995, 768)
--------------------------------------------------------------------------------
학습용 리뷰 전체에 대한 임베딩 행렬 확인 : 
[[ 0.22963014 -0.30546665 -0.0558625  ...  0.3554866   0.07065215
   0.13408607]
 [ 0.10034887  0.51734966 -0.28517404 ...  0.05499696  0.08929399
   0.04660222]
 [ 0.03501115 -0.59138674 -0.5399289  ...  0.07836973  0.6025016
  -0.2706314 ]
 ...
 [ 0.16088536  0.3518497  -0.87538946 ...  0.75445354  0.28457245
   0.36457962]
 [-0.27215552 -0.08595777  0.6485945  ... -0.12068029 -0.13909394
  -0.07137752]
 [-0.33635998  0.4318327  -0.34545404 ...  0.10001148 -0.04830904
  -0.37660566]]


In [ ]:
### 평가용 데이터(document) --> 문장 임베딩 행렬 생성(1)

# 사용자 정의 함수
# def sentence_enc(sentence):
#     return model.encode(sentence)

test_embeddings = df_test.loc[:, 'document'].apply(lambda x: model.encode(x)).values

# 결과 확인하기
print(f'평가용 전체 리뷰에 대한 임베딩 행렬의 모양 : {test_embeddings.shape}')

평가용 전체 리뷰에 대한 임베딩 행렬의 모양 : (49997,)


In [ ]:
### 평가용 데이터(document) --> 문장 임베딩 행렬 생성(2)

# sentences 생성
sentences = df_test.loc[:, 'document'].values

# model.encode(sentences) 사용
test_embeddings = model.encode(sentences)

# 결과 확인하기
print(f'평가용 전체 리뷰에 대한 임베딩 행렬의 모양 : {test_embeddings.shape}')

평가용 전체 리뷰에 대한 임베딩 행렬의 모양 : (49997, 768)


In [ ]:
### 평가용 데이터에 대한 임베딩 결과 저장하기

# 파일 경로 설정
test_embeddings_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/test_embeddings.npy'

# 결과 저장하기
np.save(test_embeddings_path, test_embeddings)

In [ ]:
### 평가용 데이터에 대한 임베딩 결과 불러오기

# 파일 경로 설정
test_embeddings_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/test_embeddings.npy'

# 저장된 평가용 임베딩 행렬 불러오기
loaded_test_embeddings = np.load(test_embeddings_path, allow_pickle=True)

# 결과 확인하기
print(f'평가용 리뷰 전체에 대한 임베딩 행렬의 모양 : {loaded_test_embeddings.shape}')
print('-'*80)
print(f'평가용 리뷰 전체에 대한 임베딩 행열 확인 : \n{loaded_test_embeddings}')

평가용 리뷰 전체에 대한 임베딩 행렬의 모양 : (49997, 768)
--------------------------------------------------------------------------------
평가용 리뷰 전체에 대한 임베딩 행열 확인 : 
[[-0.04891214 -0.3438344  -0.10130484 ... -0.14603318  0.22806278
  -0.65479577]
 [ 0.2611376  -0.80607367 -0.6604893  ...  0.2817157   0.48219842
  -0.31308427]
 [ 0.28456554  0.28490198  0.09374098 ... -0.15658678  0.53721684
  -0.04801863]
 ...
 [-0.5774844   0.51906407  0.06169851 ...  0.01148296  0.5126719
   0.99342686]
 [ 0.33191264 -0.07300376 -0.22728907 ...  0.2854041  -0.4874205
  -0.03093443]
 [-0.09021662  0.4134429  -0.28334516 ...  0.13762715  0.32434598
   1.0721738 ]]


In [ ]:
### 보충 : df.col.values에 대한 확인

# python dict 자료형 데이터 생성
pop_dict = {'2022':[np.array([9436836]), np.array([3317812]), np.array([2964820]), np.array([2365619])],
            '2015':[9904312, 3448737, 2890451, 2466052],
            '2010':[9631482, 3393191, 2632035, 2431774]}

# DataFrame 생성
df = pd.DataFrame(data=pop_dict)

# 결과 확인하기
print(df)

print('-'*80)

# 2022년 컬럼 인덱싱 --> df.col.values
print(df.loc[:, '2022'].values)

        2022     2015     2010
0  [9436836]  9904312  9631482
1  [3317812]  3448737  3393191
2  [2964820]  2890451  2632035
3  [2365619]  2466052  2431774
--------------------------------------------------------------------------------
[array([9436836]) array([3317812]) array([2964820]) array([2365619])]


In [ ]:
### X_train 생성
X_train = pd.DataFrame(data=loaded_train_embeddings)
print(X_train)

             0         1         2         3         4         5         6    \
0       0.229630 -0.305467 -0.055862 -0.810149  0.295727 -1.007614 -0.658901   
1       0.100349  0.517350 -0.285174  0.700975 -0.242628 -0.459034 -0.250942   
2       0.035011 -0.591387 -0.539929  0.306152 -0.052974 -0.767612 -0.155208   
3       0.136597 -0.102633  0.427375  0.192248 -0.002059  0.413561  0.462351   
4      -0.390766 -0.215280  0.269350  0.157344 -0.430786 -0.232554 -0.489062   
...          ...       ...       ...       ...       ...       ...       ...   
149990 -0.371298 -0.146962  0.785210  0.437134 -0.018172  0.148512  0.532089   
149991  0.222332  0.079697 -0.505798  0.067344 -0.052917 -0.058388 -0.149334   
149992  0.160885  0.351850 -0.875389  0.616683  0.661793 -0.640041  0.424856   
149993 -0.272156 -0.085958  0.648594  0.569168  0.259150  0.845531 -0.638321   
149994 -0.336360  0.431833 -0.345454 -0.314272 -0.244533  0.343347 -0.345283   

             7         8         9    .

In [ ]:
### y_train 생성
y_train = df_train.loc[:, 'label']
print(y_train)

0         0
1         1
2         0
3         0
4         1
         ..
149990    0
149991    1
149992    0
149993    1
149994    0
Name: label, Length: 149995, dtype: int64


In [ ]:
### X_test 생성
X_test = pd.DataFrame(data=loaded_test_embeddings)
print(X_test)

            0         1         2         3         4         5         6    \
0     -0.048912 -0.343834 -0.101305 -0.078701  0.054339 -1.238898 -0.488888   
1      0.261138 -0.806074 -0.660489  0.339362 -0.367431 -0.941752 -0.066228   
2      0.284566  0.284902  0.093741 -0.017227  0.061789 -0.239323  0.268976   
3     -0.016888 -0.335148 -0.017589  0.053198 -0.579792 -0.185808  0.243329   
4      0.055227  0.631026  0.129269 -0.504613  0.160170  0.039439  0.226791   
...         ...       ...       ...       ...       ...       ...       ...   
49992  0.562343  0.006787  0.393914 -0.142192 -0.193313  0.614337 -0.238424   
49993 -0.413807  0.090225 -0.302836 -0.476721  0.023064  0.228728 -0.055091   
49994 -0.577484  0.519064  0.061699  0.202573  0.003623  0.164711 -0.315304   
49995  0.331913 -0.073004 -0.227289  0.235991 -0.669331  0.198543 -0.422011   
49996 -0.090217  0.413443 -0.283345  0.265301  0.437998 -0.734049 -0.478744   

            7         8         9    ...       758 

In [ ]:
### y_test 생성
y_test = df_test.loc[:, 'label']
print(y_test)

0        1
1        0
2        0
3        0
4        0
        ..
49992    1
49993    0
49994    0
49995    0
49996    0
Name: label, Length: 49997, dtype: int64


## LightGBM 모델을 이용한 텍스트 감성 분류

### 모델 생성

In [ ]:
# 필요한 함수 임폴트
from lightgbm import LGBMClassifier

# 모델 생성
lgbm = LGBMClassifier(learning_rate=0.1, n_estimators=300, random_state=0)

### 모델 학습

In [ ]:
lgbm.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 74825, number of negative: 75170
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.873014 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 149995, number of used features: 768
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498850 -> initscore=-0.004600
[LightGBM] [Info] Start training from score -0.004600


LGBMClassifier(n_estimators=300, random_state=0)

### 평가용 데이터를 이용한 예측

In [ ]:
pred_test = lgbm.predict(X_test)

### 모델 평가

In [ ]:
### 평가 함수 정의

# 필요한 함수 임폴트
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# 평가 함수 정의
def eval_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    # 각 평가 결과 --> 데이터프레임 생성
    data = [[accuracy, f1, precision, recall]]
    columns = ['accuracy', 'f1_score', 'precision', 'recall']
    df_eval = pd.DataFrame(data=data, columns=columns)

    return df_eval

In [ ]:
### 평가용 데이터에 대한 성능 평가
result = eval_model(y_true=y_test, y_pred=pred_test)

# 결과 확인하기
print(f'평가용 데이터에 대한 모델 성능 평가 : \n{result}')

평가용 데이터에 대한 모델 성능 평가 : 
   accuracy  f1_score  precision    recall
0   0.83905  0.839449   0.843166  0.835763
